In [ ]:
# Thanks for Code from https://www.kaggle.com/mirzarahim/introduction-to-pca-image-compression-example
# Introduction to PCA: Image Compression example

# https://github.com/vivekrmk/Image-Compression-Principal-Component-Analysis-Pytorch/blob/main/Pytorch_PCA_journey.ipynb
# https://github.com/Erikfather/PCA-python/blob/master/Face_Rec.py

In [113]:
# Checking GPU Units

import torch
torch.manual_seed(0)
torch.backends.cudnn.deterministic = False
torch.backends.cudnn.benchmark = True

import torchvision
import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data.dataset import Dataset

print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))
print(torch.cuda.device_count())

True
NVIDIA GeForce RTX 2060
1


In [114]:
DATA_PATH = '../../../Documents/shopee-product-matching/'

import numpy as np
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2, matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
from matplotlib.image import imread
from IPython.display import Image
from PIL import Image
import tensorflow as tf
from tqdm import tqdm

train = pd.read_csv(DATA_PATH + 'train.csv')
train['image'] = DATA_PATH + 'train_images/' + train['image']

n_train = 12000
n_test = 1000
n_valid = 300
K = 120 # PCA, num of principal components

sample = train.head(n_train)
test   = train.loc[n_train+1:n_train+n_test]
test = test.reset_index(drop=True) # initialize indexing
image_idx = sample['image']


In [115]:
class ImageTransform():
    def __init__(self, mean, std):
        self.data_transform = transforms.Compose([
            transforms.Resize((256,256)),
            transforms.ToTensor(),
            transforms.Normalize(mean, std)
        ])
    def __call__(self, img):
        return self.data_transform(img)

In [116]:
class Img_Dataset(torch.utils.data.Dataset):
    def __init__(self, file_list, transform):
        self.file_list = file_list
        self.transform = transform
        
    def __len__(self):
        return len(self.file_list)
    
    def __getitem__(self, index):
        img_path = self.file_list[index]
        img = Image.open(img_path)
        img = img.convert('L')
        img_transformed = self.transform(img)
        
        return img_transformed

In [117]:
# Preparing train dataset, 2021. 5. 19
train_img_list = image_idx

mean = (0.0,)
std = (1.0,)

train_dataset = Img_Dataset(file_list = train_img_list,
                            transform=ImageTransform(mean, std))
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=False)

batch_iterator = iter(train_dataloader)
images = next(batch_iterator)

print(images.size())

# torch.Size[Batch Size, Channel, Width, Height]

torch.Size([16, 1, 256, 256])


In [118]:
# prepare test features 2021. 5. 19
test_image_idx = test['image']
test_img_list = test_image_idx

test_dataset = Img_Dataset(file_list = test_img_list,
                            transform=ImageTransform(mean, std))
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=16, shuffle=False)

batch_iterator = iter(test_dataloader)
test_images = next(batch_iterator)

len(batch_iterator)
#print(test_images.size())

63

In [120]:
# Making **train** features, 2021. 5. 19 
DEVICE = 'cuda'

train_feature = []
train_feature = torch.tensor(train_feature)

# transfer to DEVICE (GPU memory)
train_feature = train_feature.to(DEVICE)

a = 1
with torch.no_grad():
    for batch in tqdm(train_dataloader):
        batch = batch.to(DEVICE)
        batch = batch.permute(0,2,3,1)[:,:,:,0]
        idx, row, col = batch.shape
        batch = batch.view([len(batch),-1])

        U,S,V = torch.pca_lowrank(batch, q=len(batch), center=True, niter=3)
        V = torch.tensor(V)
        #print('U shape : ', U.shape)
        #print('S shape : ', S.shape)
        #print('V shape : ', V.shape)
        
        train_feature = torch.cat([train_feature, V.T[:,:K]], dim = 0)
        #print('train_feature shape: ', train_feature.shape)
        
        # For debugging (breaking)
        #if a == 3:
        #    break
        #print('iter num : ', a)
        a = a + 1
        

  0%|                                                                                          | 0/750 [00:00<?, ?it/s]<ipython-input-120-82a027b5f120>:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  V = torch.tensor(V)
100%|████████████████████████████████████████████████████████████████████████████████| 750/750 [04:37<00:00,  2.70it/s]


In [121]:
# Saving **train** Features 2021. 5. 19
train_feature = train_feature.data.cpu().numpy()
np.savetxt('trained_feature.csv', train_feature, delimiter=",")

In [160]:
# Loading **train** Features 2021. 5. 19
train_feature = np.loadtxt('trained_feature.csv', delimiter=",")
train_feature = torch.from_numpy(train_feature)
train_feature = train_feature.to(DEVICE)

# l2 norm to kill all the sim in 0-1   ** train_feature
from sklearn.preprocessing import normalize
train_feature = train_feature.data.cpu().numpy()
train_feature = np.vstack(train_feature)
train_feature = normalize(train_feature)
train_feature = torch.from_numpy(train_feature)
train_feature = train_feature.to(DEVICE)

In [170]:
# Making **test** features, 2021. 5. 19 
DEVICE = 'cuda'

test_feature = []
test_feature = torch.tensor(test_feature)

# transfer to DEVICE (GPU memory)
test_feature = test_feature.to(DEVICE)

a = 1
with torch.no_grad():
    for batch in tqdm(test_dataloader):
        batch = batch.to(DEVICE)
        batch = batch.permute(0,2,3,1)[:,:,:,0]
        idx, row, col = batch.shape
        batch = batch.view([len(batch),-1])

        U,S,V = torch.pca_lowrank(batch, q=len(batch), center=True, niter=3)
        V = torch.tensor(V)
        #print('U shape : ', U.shape)
        #print('S shape : ', S.shape)
        #print('V shape : ', V.shape)
        
        test_feature = torch.cat([test_feature, V.T[:,:K]], dim = 0)
        #print('test_feature shape: ', test_feature.shape)
        
        # For debugging (breaking)
        #if a == 3:
        #    break
        #print('iter num : ', a)
        a = a + 1
        

  0%|                                                                                           | 0/63 [00:00<?, ?it/s]<ipython-input-170-54f1199d7145>:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  V = torch.tensor(V)
100%|██████████████████████████████████████████████████████████████████████████████████| 63/63 [00:20<00:00,  3.05it/s]


In [171]:
# Saving **test** Features 2021. 5. 19
test_feature = test_feature.data.cpu().numpy()
np.savetxt('test_feature.csv', test_feature, delimiter=",")

In [172]:
# Loading **test** Features 2021. 5. 19
test_feature = np.loadtxt('test_feature.csv', delimiter=",")
test_feature = torch.from_numpy(test_feature)
test_feature = test_feature.to(DEVICE)

# l2 norm to kill all the sim in 0-1    ** test_feature
test_feature = test_feature.data.cpu().numpy()
test_feature = np.vstack(test_feature)
test_feature = normalize(test_feature)
test_feature = torch.from_numpy(test_feature)
test_feature = test_feature.to(DEVICE)

In [168]:
# Checking train_feature with train_feature, 2021. 5. 19
preds = []
CHUNK = 100

print('Finding similar images...')
CTS = len(train_feature)//CHUNK
if len(train_feature)%CHUNK != 0:
    CTS += 1
    
for j in tqdm(range(CTS)):
    a = j*CHUNK
    b = (j+1)*CHUNK
    b = min(b, len(train_feature))
    #print('chunk', a, 'to', b)
    
    #distances = torch.cdist(train_feature, train_feature[a:b], p=2.0).T
    distances = torch.matmul(train_feature, train_feature[a:b].T).T
    distances = distances.data.cpu().numpy()
    
    #print(type(distances))
    #print(distances.shape)
    '''
    for k in range(b-a):
        IDX = np.argmin(distances[k][:])
        o = sample.iloc[IDX].label_group
        preds.append(o)
    '''
    
    for k in range(b-a):
        #IDX = np.argmax(distances[k][:])
        IDX = np.where(distances[k,]>0.95)[0][:]
        o = sample.iloc[IDX].label_group.values
        preds.append(o)
        #print(len(IDX))
    
sample['predicted_label'] = preds


  2%|██                                                                                | 3/120 [00:00<00:05, 21.80it/s]

Finding similar images...


100%|████████████████████████████████████████████████████████████████████████████████| 120/120 [00:05<00:00, 22.28it/s]
<ipython-input-168-676261c3d217>:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample['predicted_label'] = preds


In [163]:
distances

array([[-2.68246355e-01, -1.16132840e-01,  4.19594653e-01, ...,
        -5.86927546e-01, -6.87523003e-02, -1.83017574e-02],
       [-3.34991036e-01, -4.53910072e-01,  1.15669825e-01, ...,
         4.49153745e-01,  1.70050367e-01,  2.68728962e-02],
       [-4.79847594e-01, -2.99720544e-01,  6.53674125e-01, ...,
        -7.24444193e-01,  1.47490097e-02, -4.69505347e-04],
       ...,
       [ 4.60027185e-01,  3.93653413e-01, -5.54306925e-01, ...,
         1.00000000e+00, -1.00492205e-02, -2.18378773e-01],
       [-2.01583515e-01, -1.55051082e-01,  2.28743321e-01, ...,
        -1.00492205e-02,  1.00000000e+00,  1.50841058e-01],
       [-4.92016045e-02, -8.76071869e-02,  3.77584134e-02, ...,
        -2.18378773e-01,  1.50841058e-01,  1.00000000e+00]])

In [169]:
# Scoring
# Calculate Precision
correct = 0
for i in range(len(sample)):
    if len( np.intersect1d(sample['predicted_label'][i], sample['label_group'][i])) == 1:
        correct = correct + 1

precision = correct/len(sample) * 100
print('num of correct : ', correct)
print('precision : ', precision)

# Calculate Recall
correct = 0
recall = 0
temp = 0

for i in range(len(sample)):
    if len( np.intersect1d(sample['predicted_label'][i], sample['label_group'][i])) == 1:
        L = len(sample['predicted_label'][i])
        correct = correct + 1
        temp = 1/L
        recall = recall + temp

recall = recall / correct
print('recall : ', recall)

# Calculate F1 score
print('f1 : ', 2*(precision * recall)/(precision + recall))

num of correct :  12000
precision :  100.0
recall :  0.47925798564744226
f1 :  0.9539441179310858


In [186]:
# Checking train_feature with test_feature, 2021. 5. 19
preds = []
CHUNK = 100

print('Finding similar images...')
CTS = len(test_feature)//CHUNK
if len(test_feature)%CHUNK != 0:
    CTS += 1
    
for j in tqdm(range(CTS)):
    a = j*CHUNK
    b = (j+1)*CHUNK
    b = min(b, len(test_feature))
    #print('chunk', a, 'to', b)
    
    #distances = torch.cdist(train_feature, test_feature[a:b], p=2.0).T
    distances = torch.matmul(train_feature, test_feature[a:b].T).T
    distances = distances.data.cpu().numpy()
    
    #print(type(distances))
    #print(distances.shape)
    
    for k in range(b-a):
        #IDX = np.argmax(distances[k][:])
        IDX = np.where(distances[k,]>0.9)[0][:]
        o = sample.iloc[IDX].label_group.values
        preds.append(o)
        #print(len(IDX))
        
test['predicted_label'] = preds


 20%|████████████████▌                                                                  | 2/10 [00:00<00:00, 18.74it/s]

Finding similar images...


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 19.17it/s]


In [187]:
len(test['predicted_label'][0])

3151

In [188]:
# Scoring
# Calculate Precision
correct = 0
for i in range(len(test)):
    if len( np.intersect1d(test['predicted_label'][i], test['label_group'][i])) == 1:
        correct = correct + 1

precision = correct/len(test) * 100
print('num of correct : ', correct)
print('precision : ', precision)

# Calculate Recall
correct = 0
recall = 0
temp = 0

for i in range(len(test)):
    if len( np.intersect1d(test['predicted_label'][i], test['label_group'][i])) == 1:
        L = len(test['predicted_label'][i])
        correct = correct + 1
        temp = 1/L
        recall = recall + temp

recall = recall / correct
print('recall : ', recall)

# Calculate F1 score
print('f1 : ', 2*(precision * recall)/(precision + recall))

num of correct :  95
precision :  9.5
recall :  0.0005024940435086099
f1 :  0.0010049349318785481
